In [54]:
# get clients and resources
import boto3
ec2 = boto3.resource('ec2')
elb = boto3.client('elb')
route53 = boto3.client('route53')

In [55]:
# Create the security group for the load balancer
balancer_security_group_name = 'balancer-security-group'
balancer_security_group = ec2.create_security_group(
    GroupName=balancer_security_group_name,
    Description='Security group for the load balancer'
)
response = balancer_security_group.authorize_ingress(
    IpPermissions=[
        {
            'FromPort': 80,
            'ToPort': 80,
            'IpProtocol': 'tcp',
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

In [56]:
# Create the security group for the instances
instances_security_group_name = 'instances-security-group'
instances_security_group = ec2.create_security_group(
    GroupName=instances_security_group_name,
    Description='Security group for the instances'
)
response = instances_security_group.authorize_ingress(
    IpPermissions=[
        {
            'FromPort': 4567,
            'ToPort': 4567,
            'IpProtocol': 'tcp',
            'UserIdGroupPairs': [
                {
                    'GroupId': balancer_security_group.group_id,
                    'UserId': balancer_security_group.owner_id
                }
            ]
        }
    ]
)

In [ ]:
# Create a list of availability zones for the load balancer
subnet_iterator = ec2.subnets.filter(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                balancer_security_group.vpc_id
            ]
        }
    ]
)
availability_zones = [subnet.availability_zone for subnet in subnet_iterator]

In [ ]:
# Create the classic load balancer
load_balancer_name = 'classic-load-balancer'
response = elb.create_load_balancer(
    LoadBalancerName=load_balancer_name,
    Listeners=[
        {
            'Protocol': 'HTTP',
            'LoadBalancerPort': 80,
            'InstanceProtocol': 'HTTP',
            'InstancePort': 4567
        }
    ],
    AvailabilityZones=availability_zones,
    SecurityGroups=[
        balancer_security_group.group_id
    ]
)

In [ ]:
# Configure health checks
response = elb.configure_health_check(
    LoadBalancerName=load_balancer_name,
    HealthCheck={
        'Target': 'HTTP:4567/health',
        'Interval': 10,
        'Timeout': 5,
        'UnhealthyThreshold': 2,
        'HealthyThreshold': 5
    }
)

In [ ]:
# We need to get the id of the image we are going to use.
# We will assume that it already exists
image_iterator = ec2.images.filter(
    Filters=[
        {
            'Name': 'name',
            'Values': [
                'java-application-ami'
            ]
        }
    ]
)
image = list(image_iterator)[0]

In [ ]:
#  Launch the instances
instance_count = 2
instances = ec2.create_instances(
    ImageId=image.image_id,
    InstanceType='t2.micro',
    MaxCount=instance_count,
    MinCount=instance_count,
    SecurityGroups=[
        instances_security_group_name
    ]
)

In [ ]:
# Add name tags to the instances and create a list of instance ids
# to register the instances with the load balancer.
instance_name = 'java-application'
instance_ids = []
for instance, instance_number in zip(instances, range(instance_count)):

    # Wait for instance to start and reload attributes.
    instance.wait_until_running()
    instance.reload()

    # Add name tag to the instance
    instance.create_tags(
        Tags=[
            {
                'Key': 'Name',
                'Value': instance_name + '-' + str(instance_number)
            }
        ]
    )

    # Add the instance id to the instance id list
    instance_ids.append(
        {
            'InstanceId': instance.instance_id
        }
    )


In [ ]:
# Register the instances with the load balancer
response = elb.register_instances_with_load_balancer(
    LoadBalancerName=load_balancer_name,
    Instances=instance_ids
)
clb_waiter = elb.get_waiter('instance_in_service')
clb_waiter.wait(
    LoadBalancerName=load_balancer_name,
    Instances=instance_ids
)

In [ ]:
# Get the dns name and zone id for the load balancer
response = elb.describe_load_balancers(
    LoadBalancerNames=[
        load_balancer_name
    ]
)
load_balancer_description = response['LoadBalancerDescriptions'][0]
balancer_dns_name = load_balancer_description['DNSName']
balancer_zone_id = load_balancer_description['CanonicalHostedZoneNameID']

In [ ]:
# add the public ip address to route53 so SSH is easier
response = route53.list_hosted_zones_by_name(
    DNSName='doug-nicholson.net'
)
zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.change_resource_record_sets(
    HostedZoneId=zone_id,
    ChangeBatch={
        'Changes': [
            {
                'Action': 'CREATE',
                'ResourceRecordSet': {
                    'Name': 'clb.doug-nicholson.net',
                    'Type': 'A',
                    'AliasTarget': {
                        'HostedZoneId': balancer_zone_id,
                        'DNSName': balancer_dns_name,
                        'EvaluateTargetHealth': False
                    }
                }
            }
        ]
    }
)

# it seems like waiting will be the right thing to do
route53_waiter = route53.get_waiter('resource_record_sets_changed')
route53_waiter.wait(
    Id=response['ChangeInfo']['Id'][8:]
)

In [ ]:
# Success!
print('classic load balancer launched')